# Multicollinearity Evaluation


The result we got in the modeling process was disappointing; the performance reflected on the R-square score was low, and it is not proper to be brought forward to the production phase.

So we will try to evaluate the dataset to get a better result.


## Importing Libaries and Loading Dataset

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load the dataset
df = pd.read_csv('/home/er_bim/productivity-prediction/notebooks/data/worker_productivity_processed.csv')

# Model Retraining with Features Reduction



## Checking the Correlation Matrix

In [3]:
# print the correlation matrix
corr = df.corr(numeric_only=True).round(3)
corr.style.background_gradient(cmap='coolwarm')

,team,targeted_productivity,smv,wip,over_time,incentive,no_of_style_change,no_of_workers,actual_productivity
team,1.000000,0.073000,-0.103000,0.021000,-0.117000,0.013000,-0.018000,-0.065000,-0.099000
targeted_productivity,0.073000,1.000000,-0.096000,-0.088000,-0.076000,0.183000,-0.226000,-0.104000,0.411000
smv,-0.103000,-0.096000,1.000000,0.177000,0.689000,0.624000,0.317000,0.901000,-0.112000
wip,0.021000,-0.088000,0.177000,1.000000,0.344000,0.365000,-0.020000,0.278000,0.082000
over_time,-0.117000,-0.076000,0.689000,0.344000,1.000000,0.584000,0.057000,0.754000,-0.014000
incentive,0.013000,0.183000,0.624000,0.365000,0.584000,1.000000,0.036000,0.729000,0.271000
no_of_style_change,-0.018000,-0.226000,0.317000,-0.020000,0.057000,0.036000,1.000000,0.317000,-0.179000
no_of_workers,-0.065000,-0.104000,0.901000,0.278000,0.754000,0.729000,0.317000,1.000000,-0.022000
actual_productivity,-0.099000,0.411000,-0.112000,0.082000,-0.014000,0.271000,-0.179000,-0.022000,1.000000


There are some predictors that highly-correlated between them, thus indicates the multicollinearity.

Let's check their Variance Influence Factor (VIF) value.

In [4]:
# import the libraries
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

# run the VIF calculation on df, with column 'actual_productivity' as target
y, X = dmatrices('actual_productivity ~ targeted_productivity+smv+wip+over_time+incentive+no_of_style_change+no_of_workers',
                 data=df, return_type='dataframe')

# create the report tabel
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

,VIF,features
0,146.486641,Intercept
1,1.235793,targeted_productivity
2,5.538228,smv
3,1.272798,wip
4,2.660465,over_time
5,2.875632,incentive
6,1.351204,no_of_style_change
7,9.014431,no_of_workers


There are two features that have VIF more than 5, which number indicates the feature caused multicollinearity in the dataset.

Let's try to exclude column `no_of workers` first and recalculate the VIF values.

In [5]:
# run the VIF calculation on df, with column 'actual_productivity' as target and column 'no_of workers' excluded
y, X = dmatrices('actual_productivity ~ targeted_productivity+smv+wip+over_time+incentive+no_of_style_change',
                 data=df, return_type='dataframe')

# create the report tabel
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif

,VIF,features
0,140.963010,Intercept
1,1.202568,targeted_productivity
2,2.814492,smv
3,1.272108,wip
4,2.286510,over_time
5,2.213751,incentive
6,1.246215,no_of_style_change


The result shows that there are still some features valued more than two, that means those features is moderately-correlated at least with one other feature.

Let's recheck the correlation matrix after the columns with VIF value >2 are dropped. 

In [6]:
# drop the column "no_of_workers"
df.drop(columns=['no_of_workers', 'smv', 'incentive'], inplace=True)

# print the correlation matrix
corr = df.corr(numeric_only=True).round(3)
corr.style.background_gradient(cmap='coolwarm')

,team,targeted_productivity,wip,over_time,no_of_style_change,actual_productivity
team,1.000000,0.073000,0.021000,-0.117000,-0.018000,-0.099000
targeted_productivity,0.073000,1.000000,-0.088000,-0.076000,-0.226000,0.411000
wip,0.021000,-0.088000,1.000000,0.344000,-0.020000,0.082000
over_time,-0.117000,-0.076000,0.344000,1.000000,0.057000,-0.014000
no_of_style_change,-0.018000,-0.226000,-0.020000,0.057000,1.000000,-0.179000
actual_productivity,-0.099000,0.411000,0.082000,-0.014000,-0.179000,1.000000


In [7]:
import numpy as np
from dataclasses import dataclass

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from xgboost import XGBRegressor

## Prepare the Dataset for Modelling Process

### Separate the predictors and target

First, we separate the features in the dataset as predictors and target.

The target (y variable) feature is `actual_producticity`, thus the rest of the of the features after the feature `date` is eliminated are the predictors (X variables).

In [8]:
# set the X variables
X = df.drop(columns=['date', 'actual_productivity'],axis=1)

# set the y variable
y = df['actual_productivity']

# recheck the shape of each variable
print(f"The predictors consist of {X.shape[1]} columns and {X.shape[0]} rows.\n" )
print(f"The target column is {y.name} which consist of {y.shape[0]} rows." )

The predictors consist of 8 columns and 1108 rows.

The target column is actual_productivity which consist of 1108 rows.


In [9]:
# recheck the predictors
X.head(3)

,week,department,day,team,targeted_productivity,wip,over_time,no_of_style_change
0,Week1,sewing,Thursday,8,0.80,1108,7080,0
1,Week1,finishing,Thursday,1,0.75,802,960,0
2,Week1,sewing,Thursday,11,0.80,968,3660,0


In [10]:
# recheck the target column
y.head(3)

0    0.940725
1    0.886500
2    0.800570
Name: actual_productivity, dtype: float64

### Split the Dataset into Train and Test Sets

I will split the dataset into 70% train and 30% test.

In [11]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=22)

### Transform the predictors value

The categorical features need to be encoded to fit the model training; I chose the one-hot encoding method for this dataset.

For the numerical features, I want to do the experiment that both will be trained; the first one will be trained as the original value, and the second will be transformed first to make the value compacted. I choose the robust scaler transformation method.

In [12]:
# create instance for numerical and categorical features
num = X._get_numeric_data().columns
cat = X.drop(num, axis = 1).columns

In [13]:
# import libaries for column transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer

std_scaler = StandardScaler()
rob_scaler = RobustScaler()
cat_encoder = OneHotEncoder()

# create column transformer pipeline for original value numerical features
preprocessor1 = ColumnTransformer([("OneHotEncoder", cat_encoder, cat), 
                                  ("passthrough", "passthrough", num)]
                                )

# create column transformer pipeline for standard scaled numerical features
preprocessor2 = ColumnTransformer([("OneHotEncoder", cat_encoder, cat), 
                                  ("StandardScaler", std_scaler, num)]
                                )

# create column transformer pipeline for robust scaled numerical features
preprocessor3 = ColumnTransformer([("OneHotEncoder", cat_encoder, cat), 
                                  ("RobustScaler", rob_scaler, num)]
                                )

In [14]:
# fit and transform X_train with original value in numerical features
X_train_ori = preprocessor1.fit_transform(X_train)

# transform X_test with original value in numerical features
X_test_ori = preprocessor1.transform(X_test)

In [15]:
# fit and transform X_train with original value in numerical features
X_train_std_scaled = preprocessor2.fit_transform(X_train)

# transform X_test with original value in numerical features
X_test_std_scaled = preprocessor2.transform(X_test)

In [16]:
# fit and transform X_train with original value in numerical features
X_train_rob_scaled = preprocessor3.fit_transform(X_train)

# transform X_test with original value in numerical features
X_test_rob_scaled = preprocessor2.transform(X_test)

## Model Training

Now we try to do the modelling process of the dataset.

In [17]:
# create the function for model evaluation 
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

### Original Value Predictors

In [18]:
# create list of training algorithms
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(random_state=12),
    "Ridge": Ridge(random_state=23),
    "SVR": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=34),
    "Random Forest Regressor": RandomForestRegressor(random_state=45),
    "XGBRegressor": XGBRegressor(random_state=56),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=67)
}
model_list = []
r2_list =[]

# Define the train and test datasets
X_train=X_train_ori
X_test=X_test_ori

# display the model evaluation for each algorithm
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.1381
- Mean Absolute Error: 0.0969
- R2 Score: 0.2355
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1563
- Mean Absolute Error: 0.1103
- R2 Score: 0.2034


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.1573
- Mean Absolute Error: 0.1228
- R2 Score: 0.0084
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1754
- Mean Absolute Error: 0.1333
- R2 Score: -0.0027


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.1386
- Mean Absolute Error: 0.0993
- R2 Score: 0.2296
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1576
- Mean Absolute Error: 0.1127
- R2 Score: 0.1910


SVR
Model performance for Training set
- Root Mean Squared Error: 0.1556
- Mean Absolute Error: 0.1206
- R2 Score: 0.0291
----------------------------------
Model

In [19]:
result_original_predictors = pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
result_original_predictors

,Model Name,R2_Score
6,Random Forest Regressor,0.326226
7,XGBRegressor,0.298083
0,Linear Regression,0.203450
2,Ridge,0.191035
8,AdaBoost Regressor,0.184693
4,K-Neighbors Regressor,0.027714
1,Lasso,-0.002677
3,SVR,-0.038405
5,Decision Tree,-0.308159


### Standard Scaled Value Predictors

In [20]:
# create list of training algorithms
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(random_state=12),
    "Ridge": Ridge(random_state=23),
    "SVR": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=34),
    "Random Forest Regressor": RandomForestRegressor(random_state=45),
    "XGBRegressor": XGBRegressor(random_state=56),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=67)
}
model_list = []
r2_list =[]

# Define the train and test datasets
X_train=X_train_std_scaled
X_test=X_test_std_scaled

# display the model evaluation for each algorithm
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.1382
- Mean Absolute Error: 0.0972
- R2 Score: 0.2346
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1569
- Mean Absolute Error: 0.1110
- R2 Score: 0.1980


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.1579
- Mean Absolute Error: 0.1232
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1755
- Mean Absolute Error: 0.1333
- R2 Score: -0.0034


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.1381
- Mean Absolute Error: 0.0969
- R2 Score: 0.2355
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1564
- Mean Absolute Error: 0.1104
- R2 Score: 0.2031


SVR
Model performance for Training set
- Root Mean Squared Error: 0.1111
- Mean Absolute Error: 0.0847
- R2 Score: 0.5049
----------------------------------
Model

In [21]:
result_std_scaled_predictors = pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
result_std_scaled_predictors

,Model Name,R2_Score
6,Random Forest Regressor,0.323161
7,XGBRegressor,0.298083
2,Ridge,0.203145
0,Linear Regression,0.198043
8,AdaBoost Regressor,0.183275
4,K-Neighbors Regressor,0.178348
3,SVR,0.160034
1,Lasso,-0.003373
5,Decision Tree,-0.308159


### Robust Scaled Value Predictors

In [22]:
# create list of training algorithms
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(random_state=12),
    "Ridge": Ridge(random_state=23),
    "SVR": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(random_state=34),
    "Random Forest Regressor": RandomForestRegressor(random_state=45),
    "XGBRegressor": XGBRegressor(random_state=56),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=67)
}
model_list = []
r2_list =[]

# Define the train and test datasets
X_train=X_train_rob_scaled
X_test=X_test_rob_scaled

# display the model evaluation for each algorithm
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.1381
- Mean Absolute Error: 0.0969
- R2 Score: 0.2348
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1612
- Mean Absolute Error: 0.1136
- R2 Score: 0.1531


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.1579
- Mean Absolute Error: 0.1232
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1755
- Mean Absolute Error: 0.1333
- R2 Score: -0.0034


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.1381
- Mean Absolute Error: 0.0969
- R2 Score: 0.2355
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.1608
- Mean Absolute Error: 0.1135
- R2 Score: 0.1576


SVR
Model performance for Training set
- Root Mean Squared Error: 0.1121
- Mean Absolute Error: 0.0853
- R2 Score: 0.4965
----------------------------------
Model

In [23]:
result_rob_scaled_predictors = pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)
result_rob_scaled_predictors

,Model Name,R2_Score
6,Random Forest Regressor,0.191566
2,Ridge,0.157598
0,Linear Regression,0.153120
3,SVR,0.143969
8,AdaBoost Regressor,0.139864
4,K-Neighbors Regressor,0.117987
7,XGBRegressor,0.000211
1,Lasso,-0.003373
5,Decision Tree,-0.446869


### Conclusion

The models performance of the reduced features dataset are worse than before.

# Model Retraining with Recursive Feature Elimination

Now we try another feature selection method, the Recursive Feature Elimination (RFE).

## Prepare the Dataset for Modelling Process

In [89]:
# load the dataset
df = pd.read_csv('/home/er_bim/productivity-prediction/notebooks/data/worker_productivity_processed.csv')

First, we separate the features in the dataset as predictors and target.

The target (y variable) feature is `actual_producticity`, thus the rest of the of the features after the feature `date` is eliminated are the predictors (X variables).

In [90]:
# set the X variables
X = df.drop(columns=['date', 'actual_productivity'],axis=1)

# set the y variable
y = df['actual_productivity']

# recheck the shape of each variable
print(f"The predictors consist of {X.shape[1]} columns and {X.shape[0]} rows.\n" )
print(f"The target column is {y.name} which consist of {y.shape[0]} rows." )

The predictors consist of 11 columns and 1108 rows.

The target column is actual_productivity which consist of 1108 rows.


In [91]:
# data splitting, 70% train and 30% test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=22)

In [92]:
# create instance for numerical and categorical features
num = X._get_numeric_data().columns
cat = X.drop(num, axis = 1).columns

Then, we do the column transformer.

In [93]:
std_scaler = StandardScaler()

# create column transformer pipeline for standard scaled numerical features
preprocessor = ColumnTransformer([("OneHotEncoder", cat_encoder, cat), 
                                  ("StandardScaler", std_scaler, num)]
                                )

In [94]:
# fit and transform X_train with original value in numerical features
X_train_std_scaled = preprocessor.fit_transform(X_train)

# transform X_test with original value in numerical features
X_test_std_scaled = preprocessor.transform(X_test)

## Model Training with RFE.

The estimator in RFE feature selection is Random Forest Regressor.

We need to specify the number of features to be included in the training with RFE method.

To simplify the training process, we use the same algorithm that produce the best performance, include its parameters.

In [95]:
# import the libraries
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline

In [96]:
# define the maximum n_features
total_cols = len(df.columns) -1

# create the blank list for s-squared scores
r2_scores = []

for n_features in range(2, total_cols):
    # initialize the RFE with the specified number of features
    rfe = RFE(estimator=RandomForestRegressor(random_state=45), n_features_to_select=n_features)
    
    # set the training algorithm
    model = RandomForestRegressor(n_estimators= 300, 
                                 min_samples_split= 5, 
                                 min_samples_leaf= 4, 
                                 max_depth= 20, 
                                 criterion= 'squared_error', 
                                 random_state=45
                                 )
    
    # define the pipeline
    rf_pipeline = Pipeline(steps=[('c',rfe),('m',model)])
    
    # fit the model
    rf_pipeline.fit(X_train_std_scaled, y_train)
    
    # predict on the test set
    y_pred = rf_pipeline.predict(X_test_std_scaled)
    
    # calculate the R² score
    r2 = r2_score(y_test, y_pred)
    
    # store the result to the blank list
    r2_scores.append((n_features, r2))

    # print the R² score for the current number of features
    print(f"Number of features: {n_features}, R² score: {r2:.4f}")

Number of features: 2, R² score: 0.1986
Number of features: 3, R² score: 0.2474
Number of features: 4, R² score: 0.3050
Number of features: 5, R² score: 0.3215
Number of features: 6, R² score: 0.3720
Number of features: 7, R² score: 0.3748
Number of features: 8, R² score: 0.3918
Number of features: 9, R² score: 0.3872
Number of features: 10, R² score: 0.3899
Number of features: 11, R² score: 0.4027


The best model performance is determined by 11 features resulted from the RFE.

Now let's we try to do the cross-validation.

In [102]:
# set the model training with 11 features in RFE process
rfe = RFE(estimator=RandomForestRegressor(random_state=45), n_features_to_select=11)
model = RandomForestRegressor(n_estimators= 300, 
                                 min_samples_split= 5, 
                                 min_samples_leaf= 4, 
                                 max_depth= 20, 
                                 criterion= 'squared_error', 
                                 random_state=45
                                 )
    
# define the pipeline
rf_pipeline = Pipeline(steps=[('c',rfe),('m',model)]).fit(X_train_std_scaled, y_train)

# fit the model
#rf_pipeline.fit(X_train_std_scaled, y_train)
    
# predict on the test set
y_pred = rf_pipeline.predict(X_test_std_scaled)
    
# calculate the R² score
r2 = r2_score(y_test, y_pred)

In [100]:
# import the library
from sklearn.model_selection import cross_val_score

# predicting Cross Validation Score
cv_rf_pipeline = cross_val_score(estimator = rf_pipeline, X = X_test_std_scaled, y = y_test, cv = 5, scoring='r2')

In [101]:
print('R² score:', round( rf_pipeline.score(X_test_std_scaled, y_test),4) )
print("Cross-Validated R² score: ", round(cv_rf_pipeline.mean(),4) )

R² score: 0.4027
Cross-Validated R² score:  0.3214


## Conclusion

We have a better R² score with the RFE feature selection method.

The parameters applied in the RFE are: Random Forest Regressor estimator with 11 features.

After the features selected by RFE, then followed by train the model with Random Forest Regressor contains this paramaters: 
- n_estimators= 300, 
- min_samples_split= 5
- min_samples_leaf= 4
- max_depth= 20 
- criterion= 'squared_error'
- random_state=45

Then folowed by cross-validating the model with 5 fold to get the mean R² score.

# Model Retraining with Principal Component Analysis

Now we try the Principal Comoonent Analysis (PCA).

## Prepare the Dataset for Modelling Process

In [60]:
# load the dataset
df = pd.read_csv('/home/er_bim/productivity-prediction/notebooks/data/worker_productivity_processed.csv')

First, we separate the features in the dataset as predictors and target.

The target (y variable) feature is `actual_producticity`, thus the rest of the of the features after the feature `date` is eliminated are the predictors (X variables).

In [61]:
# set the X variables
X = df.drop(columns=['date', 'actual_productivity'],axis=1)

# set the y variable
y = df['actual_productivity']

# recheck the shape of each variable
print(f"The predictors consist of {X.shape[1]} columns and {X.shape[0]} rows.\n" )
print(f"The target column is {y.name} which consist of {y.shape[0]} rows." )

The predictors consist of 11 columns and 1108 rows.

The target column is actual_productivity which consist of 1108 rows.


In [62]:
# data splitting, 70% train and 30% test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=22)

In [63]:
# create instance for numerical and categorical features
num = X._get_numeric_data().columns
cat = X.drop(num, axis = 1).columns

Then, we do the column transformer.

In [64]:
# create column transformer pipeline for standard scaled numerical features
preprocessor = ColumnTransformer([("OneHotEncoder", cat_encoder, cat), 
                                  ("passthrough", "passthrough", num)]
                                )

In [65]:
# fit and transform X original value in numerical features
X_train_transformed = preprocessor.fit_transform(X_train)

# transform X_test with original value in numerical features
X_test_transformed = preprocessor.transform(X_test)

## Model Training with PCA.

We will try to find the n_components with the best result.

To simplify the training process, we use the same algorithm that produce the best performance, include its parameters.

In [66]:
# import the libraries
from sklearn.decomposition import PCA

In [67]:
# define the maximum n_features
total_cols = len(df.columns) -1

# create the blank list for s-squared scores
r2_scores = []

for n_components in range(2, total_cols):
    # initialize the PCA with the specified number of components
    pca = PCA(n_components=n_components, random_state=97)
    
    # fit and transfor to the predictors
    X_train_PCA = pca.fit_transform(X_train_transformed)
    X_test_PCA = pca.fit_transform(X_test_transformed)
    
    # set the training algorithm
    model = RandomForestRegressor(n_estimators= 300, 
                                 min_samples_split= 5, 
                                 min_samples_leaf= 4, 
                                 max_depth= 20, 
                                 criterion= 'squared_error', 
                                 random_state=45
                                 )
    
    # fit the model
    model.fit(X_train_PCA, y_train)
    
    # predict on the test set
    y_pred = model.predict(X_test_PCA)
    
    # calculate the R² score
    r2 = r2_score(y_test, y_pred)
    
    # store the result to the blank list
    r2_scores.append((n_features, r2))

    # print the R² score for the current number of features
    print(f"Number of components: {n_components}, R² score: {r2:.4f}")

Number of components: 2, R² score: -0.0397
Number of components: 3, R² score: 0.0708
Number of components: 4, R² score: 0.1756
Number of components: 5, R² score: 0.1639
Number of components: 6, R² score: 0.2245
Number of components: 7, R² score: 0.2735
Number of components: 8, R² score: 0.2945
Number of components: 9, R² score: 0.2775
Number of components: 10, R² score: 0.2608
Number of components: 11, R² score: 0.2599


The best model performance is determined by 11 features resulted from the RFE.

Now let's we try to do the cross-validation.

## Conclusion

The R² score with the PCA feature selection method is worse than the one resulted from the RFE method.

# Best Model

Before we decide to take the best model to the production phase, we need to try to tune the parameters one more time.

## Recall the best model

In [76]:
# recall the best model with feature selection
rfe_result = round(cv_rf_pipeline.mean(), 4)

# check the parameters used by the model
from pprint import pprint
print('Parameters used by model:\n')
pprint(rf_pipeline.get_params())

Parameters used by model:

{'c': RFE(estimator=RandomForestRegressor(random_state=45), n_features_to_select=11),
 'c__estimator': RandomForestRegressor(random_state=45),
 'c__estimator__bootstrap': True,
 'c__estimator__ccp_alpha': 0.0,
 'c__estimator__criterion': 'squared_error',
 'c__estimator__max_depth': None,
 'c__estimator__max_features': 1.0,
 'c__estimator__max_leaf_nodes': None,
 'c__estimator__max_samples': None,
 'c__estimator__min_impurity_decrease': 0.0,
 'c__estimator__min_samples_leaf': 1,
 'c__estimator__min_samples_split': 2,
 'c__estimator__min_weight_fraction_leaf': 0.0,
 'c__estimator__monotonic_cst': None,
 'c__estimator__n_estimators': 100,
 'c__estimator__n_jobs': None,
 'c__estimator__oob_score': False,
 'c__estimator__random_state': 45,
 'c__estimator__verbose': 0,
 'c__estimator__warm_start': False,
 'c__importance_getter': 'auto',
 'c__n_features_to_select': 11,
 'c__step': 1,
 'c__verbose': 0,
 'm': RandomForestRegressor(max_depth=20, min_samples_leaf=4, min

## Model Hyperparameter Tuning

In [80]:
# set the pipeline
rfe = RFE(estimator=RandomForestRegressor(random_state=45), n_features_to_select=11)
model = RandomForestRegressor(random_state=45)
    
# define the pipeline
rf_pipeline = Pipeline(steps=[('c',rfe),('m',model)])

# define the RFE parameter
features = {'c__n_features_to_select': 
# define the modelling parameters
param_grid = {
    'm__criterion': ["squared_error", "absolute_error", "friedman_mse", "poisson"],
    'm__n_estimators': [100, 200, 300],
    'm__max_depth': [10, 20, 30, None],
    'm__min_samples_split': [2, 5, 10],
    'm__min_samples_leaf': [1, 2, 4]
}

## Grid Search Cross-Validation

In [81]:
# Set up the GridSearchCV
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=rf_pipeline, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

# Define the train and test datasets
X_train=X_train_std_scaled
X_test=X_test_std_scaled

# Fit the GridSearchCV to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best parameters found: {best_params}")

# Make predictions on the test set
y_test_pred = best_model.predict(X_test)

# Evaluate the best model
r2_value_gridsearch = r2_score(y_test, y_test_pred)
print(f"Best Model Test Set R2: {r2_value_gridsearch:.4f}")

Best parameters found: {'m__criterion': 'poisson', 'm__max_depth': 20, 'm__min_samples_leaf': 4, 'm__min_samples_split': 2, 'm__n_estimators': 300}
Best Model Test Set R2: 0.4003
